# Relatório do Segundo Trabalho
***
### Disciplina: (COMP0427) Inteligência Artificial
### Professor: Dr. Hendrik Texeira Macedo
### Participantes e tarefa responsável:
- Arianne Macena (Naive Bayes)
- Adam (Naive Bayes)
- Fabiana (Decision Tree)
- Rodrigo Akio Otsuka (Decision Tree)
- Valmir (Amostragem)
- Welerson (Amostragem)

### Rede bayesiana escolhida: [ChestClinic](https://www.norsys.com/netlibrary/index.htm)
### Característica de classe: Dispnéia

## Processo de amostragem
    Algoritmo: PriorSampling
***

## Naive Bayes
***

## Tree Decision
***

## Questionário
***
1. O quanto cada modelo se aproxima da rede bayesiana original ?
    * Resposta:
1. O quanto o tamanho do dataset influencia na qualidade ?
1. Smoothing se faz necessário ?
1. Quem possui melhor comportamento: Naive Bayes ou Decision Tree ? 